# Install conda and related

https://towardsdatascience.com/conda-google-colab-75f7c867a522

In [15]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

Process is interrupted.


In [ ]:
%%bash
conda install --channel defaults conda python=3.7 --yes
conda update --channel defaults --all --yes

In [8]:
!python --version

Python 3.7.11


In [ ]:
!conda install --channel conda-forge bcolz --yes

In [13]:
# setting the environmental variables
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.7/site-packages"))

# Download the data

In [16]:
!wget https://sharehost.hms.harvard.edu/sysbio/alquraishi/proteinnet/human_readable/casp11.tar.gz

--2021-08-14 16:05:25--  https://sharehost.hms.harvard.edu/sysbio/alquraishi/proteinnet/human_readable/casp11.tar.gz
Resolving sharehost.hms.harvard.edu (sharehost.hms.harvard.edu)... 134.174.159.103
Connecting to sharehost.hms.harvard.edu (sharehost.hms.harvard.edu)|134.174.159.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11603494538 (11G) [application/x-gzip]
Saving to: ‘casp11.tar.gz’

casp11.tar.gz       100%[===================>]  10.81G   105MB/s    in 1m 50s  

2021-08-14 16:07:15 (101 MB/s) - ‘casp11.tar.gz’ saved [11603494538/11603494538]



In [17]:
!tar -xzvf casp11.tar.gz

casp11/
casp11/training_50
casp11/training_95
casp11/training_30
casp11/testing
casp11/training_90
casp11/training_70
casp11/validation
casp11/training_100


# Run the original code

In [14]:
import os
import numpy as np
from io import BytesIO
from tqdm import tqdm
import bcolz

In [19]:
pn_path = os.curdir + '/casp11/'
data_path = os.curdir 

In [ ]:
ids = []
seqs = []
evs = []
coords = []
masks = ['init', '/n']
id_next, pri_next, ev_next, ter_next, msk_next = False, False, False, False, False
with open(pn_path+'training_30') as fp:
    for line in tqdm(iter(fp.readline, '')):
        if id_next: ids.append(line[:-1])
        elif pri_next: seqs.append(line[:-1])
        elif ev_next: evs.append(np.genfromtxt(BytesIO(line.encode())))
        elif ter_next: coords.append(np.genfromtxt(BytesIO(line.encode())))
        elif msk_next: masks.append(line[:-1])
        
        if np.core.defchararray.find(line, "[ID]", end=5) != -1:
            id_next = True
            masks.pop()
            masks.pop()
            pri_next, ev_next, ter_next, msk_next = False, False, False, False
        elif np.core.defchararray.find(line, "[PRIMARY]", end=10) != -1:
            pri_next = True
            ids.pop()
            id_next, ev_next, ter_next, msk_next = False, False, False, False
        elif np.core.defchararray.find(line, "[EVOLUTIONARY]", end=15) != -1:
            ev_next = True
            seqs.pop()
            id_next, pri_next, ter_next, msk_next = False, False, False, False
        elif np.core.defchararray.find(line, "[TERTIARY]", end=11) != -1:
            ter_next = True
            evs.pop()
            id_next, pri_next, ev_next, msk_next = False, False, False, False
        elif np.core.defchararray.find(line, "[MASK]", end=7) != -1:
            msk_next = True
            coords.pop()
            id_next, pri_next, ev_next, ter_next = False, False, False, False

608064it [23:44, 286.46it/s]

In [ ]:
print('# IDs: {}'.format(len(ids)))
print('# Seqs: {}'.format(len(seqs)))
print('# PSSMs: {}'.format(len(evs)))
print('# Coords: {}'.format(len(coords)))
print('# Masks: {}'.format(len(masks[:-1]))) #-1 because of blank line at end of file

In [30]:
pssm = evs
xyz = coords

In [31]:
#loop through each evolutionary section
for i in range(len(ids)):
    #first store the id and sequence
    id = ids[i]
    seq = seqs[i]
    
    #next get the PSSM matrix for the sequence
    sp = 21*i
    ep = 21*(i+1)
    psi = np.array(pssm[sp:ep])
    pssmi = np.stack([p for p in psi], axis=1)
    
    #then get the coordinates
    sx = 3*i
    ex = 3*(i+1)
    xi = np.array(xyz[sx:ex])
    xyzi = np.stack([c for c in xi], axis=1)/100 #have to scale by 100 to match PDB
    
    #lastly convert the mask to indices
    msk_idx = np.where(np.array(list(masks[i])) == '+')[0]
    
    #bracket id or get "setting an array element with a sequence"
    zt = np.array([[id], seq, pssmi, xyzi, msk_idx])
    
    if i == 0:
        bc = bcolz.carray([zt], rootdir=data_path+'validation.bc', mode='w', expectedlen=len(ids))
        bc.flush()
    else:
        bc = bcolz.carray(rootdir=data_path+'validation.bc', mode='w')
        bc.append([zt])
        bc.flush()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [27]:
#bcolz.open(rootdir=data_path+'testing.bc')

# utils

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pandas as pd
import bcolz
from shutil import copyfile
from PIL import Image
import Bio.PDB as bio
import scipy
import torch
from keras.utils.np_utils import to_categorical

residue_letter_codes = {'GLY': 'G','PRO': 'P','ALA': 'A','VAL': 'V','LEU': 'L',
                        'ILE': 'I','MET': 'M','CYS': 'C','PHE': 'F','TYR': 'Y',
                        'TRP': 'W','HIS': 'H','LYS': 'K','ARG': 'R','GLN': 'Q',
                        'ASN': 'N','GLU': 'E','ASP': 'D','SER': 'S','THR': 'T'}

aa2ix= {'G': 0,'P': 1,'A': 2,'V': 3,'L': 4,
          'I': 5,'M': 6,'C': 7,'F': 8,'Y': 9,
          'W': 10,'H': 11,'K': 12,'R': 13,'Q': 14,
          'N': 15,'E': 16,'D': 17,'S': 18,'T': 19}

#get sequences and corresponding chain ids
def gather_sequence(pdb_id, seq_file):
    seqs=[]
    #chains=[]
    #get line numbers of pdb_id
    for ix, line in enumerate(seq_file):
        pos = np.core.defchararray.find(line, pdb_id)
        if pos > 0:
            seqs.append(seq_file[ix+1][:-1]) #cut off newline character
            #chains.append(line[pos+5]) #gives the chain letter from the line        
        
    return seqs

def create_targets(pdb_file_path):
    p = bio.PDBParser()
    s = p.get_structure('X', pdb_file_path)
    
    #first = []
    #last = []
    #coords = []
    #chains=[]
    chain_coords=[]
    
    #randomly select one model from the pdb file
    gen = s.get_models()
    l = list(gen)
    mod = l[np.random.randint(0, len(l))]
    
    #for model in s:
    for chain in mod:
        for ix,residue in enumerate(chain):
            coords = []
            #if ix == 0:
            #    first.append(residue.get_id()[1])
            if residue.get_id()[0] == ' ':
                #l = residue.get_id()[1]
                for atom in residue:
                    if atom.get_name() == "N":
                        n = atom.get_coord()
                    elif atom.get_name() == "CA":
                        ca = atom.get_coord()
                    elif atom.get_name() == "C":
                        cp = atom.get_coord()
                try: #in some cases N is missing on the first residue, so we append zeros instead
                    coords.append(np.stack([n,ca,cp], axis=0))
                    del n, ca, cp
                except:
                    #first[-1] += 1 move past the first residue and ignore it
                    pass
                    #coords.append(np.zeros(3,3))
        chain_coords.append(coords)
        #coords = []
        #chains.append(chain.get_id())
        #last.append(l)
    #final array is size 5x(no of residues)*3
    return chain_coords #, chains, first, last

def encode_sequence(sequence, onehot=True):
    vec=[]
    for chain in sequence:
        for c in chain:
            for aa, val in aa2ix.items():
                if c == aa:
                    vec.append(val)
    if onehot:
        encoding = to_categorical(vec, 20)
        return np.uint8(encoding)
    
    return np.uint8(vec)

def parse_pdb(pdb_file):
    #pdb_file = 'pdb5l6t.ent' #np.random.choice(pdb_list)
    p = bio.PDBParser()
    s = p.get_structure('X', pdb_path+pdb_file)
    
    gen = s.get_models()
    l = list(gen)
    mod = l[np.random.randint(0, len(l))] #choose random model when more than 1 exists
    
    seq_strs = []
    seq_locs = []
    for chain in mod:
        seq_str = ''
        seq_loc = []
        for residue in chain:
            if residue.get_id()[0] == ' ':
                letter_code = residue_letter_codes[residue.get_resname()]
                seq_str += letter_code
                for atom in residue:
                    seq_loc.append(atom.get_full_id()[3][1])
        seq_strs.append(seq_str)
        seq_locs.append(np.unique(seq_loc))
        
    return seq_strs, seq_locs

def align_indices(seq_strs, seq_locs, gt_seq, start_match_length=5):
    fill_indices = []
    for ix, pdb_seq in enumerate(seq_strs):
        search_seq = gt_seq[ix]
        pos = np.core.defchararray.find(search_seq, pdb_seq[:start_match_length])
        if pos < 0:
            raise ValueError('First 5 residues in pdb file have no match!')
        locs = seq_locs[ix] + (pos - seq_locs[ix][0])
        fill_indices.append(np.intersect1d(range(len(search_seq)), locs))
    
    return fill_indices

def calc_dist(atom1_coord, atom2_coord):
    return scipy.spatial.distance.euclidean(atom1_coord, atom2_coord)

def gt_dihedral_angles(pdb_file_path):
    p = bio.PDBParser()
    s = p.get_structure('X', pdb_file_path)
    calc_di = bio.vectors.calc_dihedral
    calc_ba = bio.vectors.calc_angle
    
    #torsional angles
    phi = []
    psi = []
    omega = []
    #bond angles
    ca_c_n = []
    c_n_ca = []
    n_ca_c = []
    #bond_lengths
    c_n = []
    n_ca = []
    ca_c = []
    
    for model in s:
        for chain in model:
            for ix, residue in enumerate(chain):
                for atom in residue:
                    if atom.get_name() == "N":
                        n = atom.get_vector()
                        n_coord = atom.get_coord()
                        if ix != 0:
                            psi.append(calc_di(np, cap, cp, n))
                            ca_c_n.append(calc_ba(cap, cp, n))
                            c_n.append(calc_dist(cp_coord, n_coord))
                    if atom.get_name() == "CA":
                        ca = atom.get_vector()
                        ca_coord = atom.get_coord()
                        if ix != 0:
                            omega.append(calc_di(cap, cp, n, ca))
                            c_n_ca.append(calc_ba(cp, n, ca))
                            n_ca.append(calc_dist(n_coord, ca_coord))
                    if atom.get_name() == "C":
                        c = atom.get_vector()
                        c_coord = atom.get_coord()
                        if ix != 0:
                            phi.append(calc_di(cp, n, ca, c))
                            n_ca_c.append(calc_ba(n, ca, c))
                            ca_c.append(calc_dist(ca_coord, c_coord))
                #store previous vectors
                np, cap, cp = n, ca, c
                cp_coord = c_coord

    torsional_angles = torch.stack([torch.tensor(psi), 
                                    torch.tensor(omega), 
                                    torch.tensor(phi)], dim=1)
    
    bond_angles = torch.stack([torch.tensor(ca_c_n), 
                               torch.tensor(c_n_ca), 
                               torch.tensor(n_ca_c)], dim=1)
    
    bond_lengths = torch.stack([torch.tensor(c_n), 
                                torch.tensor(n_ca), 
                                torch.tensor(ca_c)], dim=1)
        
    return torsional_angles, bond_angles, bond_lengths

def subset(array_path, max_len, min_len=0, save_path=None):
    """Return a subset of a bcolz array based on max and min lengths of sequences
    array_path: path to the bcolz array
    max_len: maximum length of sequences to include in subset
    min_len: Default 0, minimum length of sequences to include in subset
    save_path: Default None, path to save subset array
    """
    a = bcolz.carray(rootdir=array_path)
    
    shix = []
    for ix in range(len(a)):
        name, sequence, coords = a[ix]
        length = len(sequence[0])
        if (length >= min_len) and (length <= max_len):
            shix.append(ix)
    
    if save_path:
        subset = bcolz.carray(a[shix], rootdir=save_path, mode='w')
        subset.flush()
        return subset
    
    return a[shix]

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]


# RGN

In [ ]:
import numpy as np
import ipywidgets as ip
from matplotlib import pyplot as plt
import os
from tqdm import tqdm
from collections import Counter as cs
import sys
import Bio.PDB as bio
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim
import pdb
%matplotlib inline

In [ ]:
from data import ProteinNet, sequence_collate
from model import *
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [ ]:
from data import ProteinNet, sequence_collate
from model import *
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

## dataloader

In [ ]:
#download data from github and run the proteinnet notebook first
trn_dataset = ProteinNet(data_path+'train30.bc')
val_dataset = ProteinNet(data_path+'validation.bc')

In [ ]:
trn_data = DataLoader(trn_dataset, batch_size=32, shuffle=True, collate_fn=sequence_collate)
val_data = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=sequence_collate)

In [ ]:
#there should be exactly 3 coordinates for each residue
for i_batch, sample_batched in enumerate(trn_data):
    vec = sample_batched['sequence']
    print(i_batch, sample_batched['sequence'].size(),
         sample_batched['coords'].size())
    if i_batch == 3:
        break

## RGN model

In [ ]:
class RGN(nn.Module):
    def __init__(self, hidden_size, num_layers, linear_units=20, input_size=42):
        super(RGN, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.linear_units = linear_units
        self.grads = {}
        
        self.lstm = nn.LSTM(self.input_size, hidden_size, num_layers, bidirectional=True)
        
        #initialize alphabet to random values between -pi and pi
        u = torch.distributions.Uniform(-3.14, 3.14)
        self.alphabet = nn.Parameter(u.rsample(torch.Size([linear_units, 3])))
        self.linear = nn.Linear(hidden_size*2, linear_units)
        
        #set coordinates for first 3 atoms to identity matrix
        self.A = torch.tensor([0., 0., 1.])
        self.B = torch.tensor([0., 1., 0.])
        self.C = torch.tensor([1., 0., 0.])

        #bond length vectors C-N, N-CA, CA-C
        self.avg_bond_lens = torch.tensor([1.329, 1.459, 1.525])
        #bond angle vector, in radians, CA-C-N, C-N-CA, N-CA-C
        self.avg_bond_angles = torch.tensor([2.034, 2.119, 1.937])

    
    def forward(self, sequences, lengths):
        max_len = sequences.size(0)
        batch_sz = sequences.size(1)
        lengths = torch.tensor(lengths, dtype=torch.long, requires_grad=False)
        order = [x for x,y in sorted(enumerate(lengths), key=lambda x: x[1], reverse=True)]
        conv = zip(range(batch_sz), order) #for unorder after LSTM
        
        #add absolute position of residue to the input vector
        abs_pos = torch.tensor(range(max_len), dtype=torch.float32).unsqueeze(1)
        abs_pos = (abs_pos * torch.ones((1, batch_sz))).unsqueeze(2) #broadcasting
        
        h0 = Variable(torch.zeros((self.num_layers*2, batch_sz, self.hidden_size)))
        c0 = Variable(torch.zeros((self.num_layers*2, batch_sz, self.hidden_size)))
        
        #input needs to be float32 and require grad
        sequences = torch.tensor(sequences, dtype=torch.float32, requires_grad=True)
        pad_seq = torch.cat([sequences, abs_pos], 2)
    
        packed = pack_padded_sequence(pad_seq[:, order], lengths[order], batch_first=False)
        
        lstm_out, _ = self.lstm(packed, (h0,c0))
        unpacked, _ = pad_packed_sequence(lstm_out, batch_first=False, padding_value=0.0)
        
        #reorder back to original to match target
        reorder = [x for x,y in sorted(conv, key=lambda x: x[1], reverse=False)]
        unpacked = unpacked[:, reorder]

        #for example, see https://bit.ly/2lXJC4m
        softmax_out = F.softmax(self.linear(unpacked), dim=2)
        sine = torch.matmul(softmax_out, torch.sin(self.alphabet))
        cosine = torch.matmul(softmax_out, torch.cos(self.alphabet))
        out = torch.atan2(sine, cosine)
        
        #create as many copies of first 3 coords as there are samples in the batch
        broadcast = torch.ones((batch_sz, 3))
        pred_coords = torch.stack([self.A*broadcast, self.B*broadcast, self.C*broadcast])
        
        for ix, triplet in enumerate(out[1:]):
            pred_coords = geometric_unit(pred_coords, triplet, 
                                         self.avg_bond_angles, 
                                         self.avg_bond_lens)
        #pred_coords.register_hook(self.save_grad('pc'))
        
            
        #pdb.set_trace()
        return pred_coords
    
    def save_grad(self, name):
        def hook(grad): self.grads[name] = grad
        return hook

In [ ]:
#make sure output size and target sizes are the same
for i_batch, sampled_batch in enumerate(trn_data):
    inp_seq = sampled_batch['sequence']
    inp_lens = sampled_batch['length']
    rgn = RGN(20, 1, 20)
    out = rgn(inp_seq, inp_lens)
    print(i_batch, inp_seq.size(), sampled_batch['coords'].size(), out.size())
    
    if i_batch == 3:
        break

## training

In [ ]:
#hyperparameters are directly from the paper's author
rgn = RGN(800, 2, linear_units=60)
#rgn.load_state_dict(torch.load(data_path+'models/rgn.pt')) #load pretrained model
optimizer = torch.optim.Adam(rgn.parameters(), lr=1e-3)
drmsd = dRMSD()

In [ ]:
running_loss = 0.0

for epoch in range(30):
    last_batch = len(trn_data) - 1
    for i, data in tqdm(enumerate(trn_data)):
        names = data['name']
        coords = data['coords']
        mask = data['mask']

        optimizer.zero_grad()
        outputs = rgn(data['sequence'], data['length'])

        loss = drmsd(outputs, coords, mask)

        loss.backward()
        nn.utils.clip_grad_norm_(rgn.parameters(), max_norm=50)
        optimizer.step()

        running_loss += loss.item()
        if (i != 0) and (i % last_batch == 0):
            print('Epoch {}, Train Loss {}'.format(epoch, running_loss/i))
            running_loss = 0.0
            break
            
    last_batch = len(val_data) - 1
    for i, data in tqdm(enumerate(val_data)):
        names = data['name']
        coords = data['coords']
        mask = data['mask']
        
        outputs = rgn(data['sequence'], data['length'])
        loss = drmsd(outputs, coords, mask)

        running_loss += loss.item()
        if (i != 0) and (i % last_batch == 0):
            print('Epoch {}, Val Loss {}'.format(epoch, running_loss/i))
            running_loss = 0.0
            
print('Finished Training')

In [ ]:
torch.save(rgn.state_dict(), data_path+'models/rgn.pt')